Import modules

In [621]:
%matplotlib inline
import pandas as pd
import os, glob
import matplotlib.pyplot as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import scipy
from scipy.stats import zscore

Set up participant number

In [622]:
participantNumber = ['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14',
                     'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28',
                     'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35']

Set up block number

In [623]:
blockNumber = ['B1', 'B2', 'B3']

Set up fabric code

In [624]:
fabricCode = ['A', 'B', 'C']


Set up episode number

In [625]:
episodeNumber = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0]

Read in compilation table

In [626]:
df = pd.read_csv('C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Total load calculations\CompilationTable.csv')

Remove irrelevent coloums from dataframe

In [627]:

df = df.drop(['Position_mm', 'File_Number', 'Reference', 'Protocol', 'Analysis', 'Date_Stamp', 'Time_Stamp', 'DataSaveBlock',
              'BlockStamp_s', 'BrickInBlock', 'Time_s', 'Adjustment_s', 'RelativeTime_s', 'Trigger_V', 'SensorA_Abs_V',
              'SensorB_Abs_V', 'SensorA_Net_V', 'SensorB_Net_V', 'ForceAtXminus47p5mm_g', 'ForceAtXplus47p5mm_g', 'Study'],
             axis=1)

Create empty data frames

In [628]:
#Create emoty data frame to 
dfindexDifferences = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 'Index Difference'])

# Create empty data frame to append touch down means to
dfTouchDown = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 'EpisodeRelativeTime_s'])

# Create empty  data frame to append lift off means to
dfLiftOff = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 'EpisodeRelativeTime_s'])

Filter data

In [629]:
dfFiltered = df.loc[~(df['Episode_Number'] ==0)] #Remove any rows where Episode_Number is equal to 0
dfFiltered = dfFiltered[dfFiltered['EpisodeRelativeTime_s'] >= 0] #Remove any rows where EpisodeRelativeTime_s is equal to or less than 0
dfFiltered= dfFiltered[dfFiltered['EpisodeRelativeTime_s'] < 7] #Remove any rows where EpisodeRelativeTime_s is equal to or greater than 7
dfFiltered = dfFiltered[dfFiltered['TotalLoad_g'] > 1] #Remove any rows where TotalLoad_g is equal to or less than 0
#dfFiltered = dfFiltered.reset_index()
#dfFiltered

Loop to calculate mean for each episode

In [630]:

for p in participantNumber:
    dfP = dfFiltered[dfFiltered['Participant'] == p]  # when value in 'Participant' column is equal to the participant value in loop
    for b in blockNumber:
        dfB = dfP[dfP['Study_Block'] == b]  # when value in 'Study_Block' column is equal to theStudy_block value in loop
        for f in fabricCode:
            dfF = dfB[dfB['Fabric'] == f]  # when value in 'Fabric' column is equal to the fabric value in loop
            for e in episodeNumber:
                dfE = dfF[dfF['Episode_Number'] == e]  # when value in 'Episode number' column is equal to the episode value in loop
                dfE = dfE.assign(TotalLoad_g_zscore = '') #create a new empty column in data fram dfE called 'TotalLoad_g_zscore'
                dfE['TotalLoad_g_zscore'] = zscore(dfE['TotalLoad_g']) # create zscore for 'TotalLoad_g' and put values in a new column of dfE named 'TotalLoad_g_zscore'
                dfFiltered_zscore = dfE.loc[~(dfE['TotalLoad_g_zscore'] <= -2)] #Filter out zscores that are less than -2
                dfFiltered_zscore = dfFiltered_zscore.loc[~(dfFiltered_zscore['TotalLoad_g_zscore'] >= 2)] #Filter out scores that are great than 2
                dfFiltered_zscore = dfFiltered_zscore.reset_index() #Resets index of pandas dataframe wihlst creating a new column 'Index' containing the old index. 
               
                dfShape = len(dfFiltered_zscore.index)
                if (dfShape <= 10): #skip over dataframes that have less than 10 rows
                    continue
                
                dfMinIndex = dfFiltered_zscore.at[0, 'index'] #find the first value in  the 'index' column
                dfSecondIndex = dfFiltered_zscore.at[1,'index'] #find the second value in the 'index' column 
                dfIndexDifferenceCalculation = dfSecondIndex - dfMinIndex #Calculate the differnece between the second and first index value
                #dfindexDifferences = dfindexDifferences.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'Index Difference': dfIndexDifferenceCalculation},ignore_index=True)
                
                if (dfIndexDifferenceCalculation > 1): 
                    touchDownTime = dfFiltered_zscore.at[1, 'EpisodeRelativeTime_s']    
                    
                else:
                    touchDownTime = dfFiltered_zscore.at[0, 'EpisodeRelativeTime_s']
                
                dfTouchDown = dfTouchDown.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'EpisodeRelativeTime_s': touchDownTime}, ignore_index=True)
                
                dfReverse = dfFiltered_zscore.iloc[::-1] #Reverse the dataframe
                dfReverse = dfReverse.reset_index(drop=True)
                dfMaxIndex = dfReverse.at[1, 'index'] #find the first value in  the 'index' column
                dfSecondMaxIndex = dfReverse.at[0,'index'] #find the second value in the 'index' column 
                dfIndexDifferenceMaxCalculation = dfSecondMaxIndex - dfMaxIndex #Calculate the differnece between the second and first index value
                
                if (dfIndexDifferenceMaxCalculation > 1): 
                    LiftOffTime = dfReverse.at[1, 'EpisodeRelativeTime_s']    
                    
                else:
                    LiftOffTime = dfReverse.at[0, 'EpisodeRelativeTime_s']
                    
                dfLiftOff = dfLiftOff.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'EpisodeRelativeTime_s': LiftOffTime}, ignore_index=True)
                

     Participant Block Fabric  Episode Number  EpisodeRelativeTime_s
0            P01    B1      A             1.0               6.022120
1            P01    B1      A             2.0               6.129291
2            P01    B1      A             3.0               5.983861
3            P01    B1      A             4.0               5.437892
4            P01    B1      A             5.0               5.492574
...          ...   ...    ...             ...                    ...
4723         P35    B3      C            14.0               5.769198
4724         P35    B3      C            15.0               6.271331
4725         P35    B3      C            16.0               6.124317
4726         P35    B3      C            17.0               6.375897
4727         P35    B3      C            18.0               5.984886

[4728 rows x 5 columns]


Export Files

In [633]:
exportindexDifferences= dfindexDifferences.to_csv(r'C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Touch Down Calculations\Index_Differences.csv', mode='a', index=None)
exporttouchDownTime= dfTouchDown.to_csv(r'C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Touch Down Calculations\Touch_Down_Time.csv', mode='a', index=None)
exportdfLiftOff= dfLiftOff.to_csv(r'C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Touch Down Calculations\Lift_Off.csv', mode='a', index=None)